In [ ]:
import sqlite3

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

from stabilvol.utility import functions as f

DATABASE = 'data/processed/trapezoidal_selection/stabilvol.sqlite'

In [ ]:
import os
print(os.getcwd())
os.path.exists(DATABASE)

In [ ]:
f.list_database_thresholds(DATABASE)

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect(DATABASE)
cur = conn.cursor()

# Query the database to get all table names
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
t1 = -0.1
t2 = -1.5

t1_string = f.stringify_threshold(t1)
t2_string = f.stringify_threshold(t2)

df = pd.read_sql_query(f'SELECT * from stabilvol_{t1_string}_{t2_string}', conn)
df.head()

In [ ]:
VOL_LIMIT = 0.05
MARKET = 'UN'
df = df.query('Volatility < @VOL_LIMIT and Market == @MARKET')
df.plot(x='Volatility', y='FHT', figsize=(12, 8), kind='scatter')

In [ ]:
def error_on_the_mean(values):
    return np.std(values)/np.sqrt(len(values))

def plot_mean_range(group):
    bins_margins = group.index.categories.left.values
    mfht = group['mean']
    error = group['standard_error']
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.plot(bins_margins, mfht)
    ax.fill_between(bins_margins, mfht - error, mfht + error, alpha=0.2)
    ax.set_xlabel('Volatility')
    ax.set_ylabel('FHT')
    ax.set_title(f'FHT vs Volatility for {MARKET} market')
    plt.show()

In [ ]:
# Define the number of bins
N_BINS = 1000

# Use qcut to bin 'Volatility' values
df['Bins'] = pd.qcut(df['Volatility'], N_BINS, duplicates='drop')

# Group by the bins and calculate the mean and standard error of 'value'
grouped = df.groupby('Bins')['FHT'].agg(['mean', error_on_the_mean, 'size'])

# Rename the columns
grouped.columns = ['mean', 'standard_error', 'count']

plot_mean_range(grouped)
print(grouped['count'])

## Idea 1: Fit a Function

In [ ]:
# Define the logistic function
def logistic(x, x0, k, ymax):
    return ymax / (1 + np.exp(-k*(x-x0)))

def custom_function(x, scale1, scale2, a, b, c, d):
    y1 = scale1 / (1 + np.exp(-a*(x-b)))
    y2 = scale2 * np.exp(-c*(x)) + d
    return y1 * y2

from scipy.stats import skewnorm
# Define the skewed Gaussian function
def skewed_gaussian(x, ymax, a, loc, scale):
    skwd = skewnorm.pdf(x, a, loc, scale)
    baseline = 15 * ymax * (1 - np.exp(-10*(x-loc)))
    return ymax*skwd + baseline

def polynomial(x, a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10):
    return a0 + a1*x + a2*x**2 + a3*x**3 + a4*x**4 + a5*x**5 + a6*x**6 + a7*x**7 + a8*x**8 + a9*x**9 + a10*x**10

function = polynomial

initial_guess = [20, 10, 0.05, 0.1]

x = grouped.index.categories.left.values
y = grouped['mean'].values
y_err = grouped['standard_error'].values

# Fit the logistic function to the data
popt, pcov = curve_fit(polynomial, x, y)

# Print the optimal parameters
print(f"x0 = {popt[0]}, k = {popt[1]}, ymax = {popt[2]}")

# Plot the original data and the fitted curve
plt.errorbar(x, y, yerr=y_err, fmt='o')
plt.plot(x, function(x, *popt), 'r')

plt.show()

In [ ]:
# Fit a polynomial to the data
coeffs = np.polyfit(x, y, 12)

# Create a polynomial function from the coefficients
fitted_poly = np.poly1d(coeffs)

# Generate y-values for the fitted curve
y_fitted = fitted_poly(x)

# Print the coefficients
print(f"Coefficients: {coeffs}")

# Plot the original data and the fitted curve
plt.scatter(x, y)
plt.plot(x, y_fitted, 'r')
plt.show()

## Idea 2: Giustify a Decent Fit

Maybe while the fit does not catch the real maximum, it can be robust to changing the number of bins.

In [ ]:
for deg in [5, 7, 10, 12, 14, 16]:
    print(f'Fitting to Degree {deg} Polynomial')
    curve_params = {}
    for nbins in [250, 300, 500, 700, 1000, 2000]:
        # Use qcut to bin 'Volatility' values
        df['Bins'] = pd.qcut(df['Volatility'], nbins, duplicates='drop')
        
        # Group by the bins and calculate the mean and standard error of 'value'
        grouped = df.groupby('Bins')['FHT'].agg(['mean', error_on_the_mean, 'size'])
        
        x = grouped.index.categories.left.values
        y = grouped['mean'].values
        
        max_value = grouped['mean'].max()
        idxmax = grouped['mean'].idxmax()
    
        # Fit a polynomial of degree 2 to the data
        coeffs = np.polyfit(x, y, deg)
        x_fit = np.linspace(0, 0.05, 1000)
        y_fitted = np.poly1d(coeffs)(x_fit)
        
        # plt.scatter(x, y)
        # plt.plot(x_fit, y_fitted, 'r')
        # plt.show()
        
        curve_params[nbins] = (max_value, idxmax, y_fitted.max(), x[y_fitted.argmax()])
        
    fig, axs = plt.subplots(3, 1, figsize=(12, 8))
    # axs[0].plot(curve_params.keys(), [x[0] for x in curve_params.values()])
    axs[0].plot(curve_params.keys(), [x[2] for x in curve_params.values()])
    axs[1].plot(curve_params.keys(), [x[3] for x in curve_params.values()])
    axs[2].scatter([x[3] for x in curve_params.values()], [x[2] for x in curve_params.values()])
    plt.show()

## Idea 3: Use a Criterion to Choose Bins

We can iterate the cutting until we find that at best N points are in each bin

In [ ]:
N = 1000
nbins = 50
def select_bins(df, max_n=1000):
    nbins = 50
    
    while True:
        # Use qcut to bin 'Volatility' values
        df['Bins'] = pd.qcut(df['Volatility'], nbins, duplicates='drop')
        
        # Group by the bins and calculate the mean and standard error of 'value'
        grouped = df.groupby('Bins')['FHT'].agg(['mean', error_on_the_mean, 'size'])
        count = grouped['size'].min()
        
        if count < max_n:
            break
        else:
            nbins += 50
    return grouped



In [ ]:
market = 'UN'
vol_limit = 1

fig, axs = plt.subplots(3, 5, figsize=(12, 8))
axs = axs.flatten()

selected_tables = f.list_database_tables(DATABASE)[:15]

for i, table_name in enumerate(selected_tables):
    
    ax = axs[i]
    
    df = pd.read_sql_query(f'SELECT * from {table_name[0]}', conn)
    df = df.query('Volatility < @vol_limit and Market == @market')
    
    grouped_data = select_bins(df)
    print(len(grouped_data))
    
    x = grouped_data.index.categories.left.values
    y = grouped_data['mean'].values
    
    y_err = grouped_data['error_on_the_mean'].values
    
    ax.plot(x, y)
    ax.fill_between(x, y - y_err, y + y_err, alpha=0.2)
    ax.set_xlim(0, 1)
    
plt.show()

In [ ]:
market = 'UW'
vol_limit = 0.5

fig, axs = plt.subplots(3, 5, figsize=(12, 8))
axs = axs.flatten()

selected_tables = f.list_database_tables(DATABASE)[:15]

for i, table_name in enumerate(selected_tables):
    
    ax = axs[i]
    
    df = pd.read_sql_query(f'SELECT * from {table_name[0]}', conn)
    df = df.query('Volatility < @vol_limit and Market == @market')
    
    grouped_data = select_bins(df)
    print(len(grouped_data))
    
    x = grouped_data.index.categories.left.values
    y = grouped_data['mean'].values
    
    y_err = grouped_data['error_on_the_mean'].values
    
    ax.plot(x, y)
    ax.fill_between(x, y - y_err, y + y_err, alpha=0.2)
    
plt.show()